In [53]:
import sys
from PyQt5 import QtCore, QtWidgets, QtWebEngineWidgets
from lxml import html as htmlRenderer
import requests
import json
from datetime import date
import datetime
from random import *

In [2]:
def render(source_url):
    """Fully render HTML, JavaScript and all."""

    import sys
    from PyQt5.QtWidgets import QApplication
    from PyQt5.QtCore import QUrl
    from PyQt5.QtWebEngineWidgets import QWebEngineView

    class Render(QWebEngineView):
        def __init__(self, url):
            self.html = None
            self.app = QApplication(sys.argv)
            QWebEngineView.__init__(self)
            self.loadFinished.connect(self._loadFinished)
            #self.setHtml(html)
            self.load(QUrl(url))
            self.app.exec_()

        def _loadFinished(self, result):
            # This is an async call, you need to wait for this
            # to be called before closing the app
            self.page().toHtml(self._callable)

        def _callable(self, data):
            self.html = data
            # Data has been stored, it's safe to quit the app
            self.app.quit()

    return Render(source_url).html

In [3]:
url="https://elpais.com/internacional/2019/08/01/actualidad/1564658691_230617.html"
renderUrl = render(url)
renderedPage = htmlRenderer.fromstring(renderUrl)

In [4]:
auxLinks = renderedPage.xpath("//span[@class='boton-contador']")

In [30]:
commentEl = auxLinks[0]
commentElVal = commentEl.get("id").split("_")
idNoticia = commentElVal[len(commentElVal)-1]
perfiloHiloId = "_{}".format(idNoticia)

In [22]:
# Get Comments Info

urlInfoComments = "https://elpais.com/ThreadeskupSimple"
rnd = random()

infoArg = {
    "action": "info",
    "th": idNoticia,
    "rnd": rnd
}

In [23]:
responseInfoComments = requests.get(urlInfoComments, infoArg)

In [24]:
responseInfoComments

<Response [200]>

In [25]:
infoComments = json.loads(responseInfoComments.text)

In [26]:
infoComments

{'info': '',
 'perfilesHilos': {'_1564664936-bca025601586bc5a00ef0c26fdd878f6': {'caducidad': 1565961151,
   'estado': 0,
   'msgs_respuesta': None,
   'num_respuestas': None,
   'numero_mensajes': 120,
   'numero_mensajes_opinadores': 16,
   'premoderacion': 1,
   'tipo': 'comentarios',
   'titulo_noticia': 'Tres%20millones%20de%20hect%C3%A1reas%20de%20bosques%20arden%20en%20Rusia',
   'url_asociada': 'http://elpais.com/internacional/2019/08/01/actualidad/1564658691_230617.html'}},
 'status': 'ok'}

In [42]:
pages = infoComments["perfilesHilos"][perfiloHiloId]["numero_mensajes"]//50
if (infoComments["perfilesHilos"][perfiloHiloId]["numero_mensajes"]%50 > 0) :
    pages = pages + 1

pages

3

In [47]:
# https://elpais.com/OuteskupSimple?s=&rnd=0.5448952094970048&th=2&msg=1564664936-bca025601586bc5a00ef0c26fdd878f6&p=2&nummsg=120&tt=1
# s=
# &rnd=0.5448952094970048
# &th=2
# &msg=1564664936-bca025601586bc5a00ef0c26fdd878f6
# &p=2
# &nummsg=120
# &tt=1
rnd = random()
commentsArgs = {
    "s": "",
    "rnd": rnd,
    "th": 1,
    "msg": idNoticia,
    "nummsg": infoComments["perfilesHilos"][perfiloHiloId]["numero_mensajes"],
    "tt": 1
}

In [48]:
urlGetComments = "https://elpais.com/OuteskupSimple"
responseComments = requests.get(urlGetComments, commentsArgs)

In [49]:
commentsResponse = json.loads(responseComments.text)

In [50]:
commentsResponse

{'errorCode': 0,
 'mensajes': [{'CopiaEnTablones': 't1-registrador',
   'autorMsgRespuesta': 'registrador',
   'borrado': 0,
   'cont_adicional': 0,
   'contenido': '<p>Tres millones de hectáreas de bosques arden en Rusia <a title="http://elpais.com/internacional/2019/08/01/actualidad/1564658691_230617.html" target="_blank" href="http://cort.as/-MEWg">http://cort.as/-MEWg</a></p>',
   'extra_json': '{"lang":"es","edition":"undefined"}',
   'hayAudio': 0,
   'hilo': '1564664936-bca025601586bc5a00ef0c26fdd878f6',
   'idMsg': '1564664936-bca025601586bc5a00ef0c26fdd878f6',
   'idMsgRespuesta': '1564664936-bca025601586bc5a00ef0c26fdd878f6',
   'level': 0,
   'opinador': 1,
   'origen': 'ep',
   'reenvio': 0,
   'tsMensaje': 1564664936,
   'usuarioOrigen': 'registrador',
   'via': 'web'},
  {'CopiaEnTablones': 't1-mapereira,t4-registrador',
   'autorMsgRespuesta': 'registrador',
   'borrado': 0,
   'cont_adicional': 0,
   'contenido': '<p>Aún no toman en serio el calentamiento global. Las pé

In [55]:

def extractComments(commentsObjectList, urlNoticia="", specialCase = False):
    print(" \t -> parsing comments list with -{}- elements:".format(len(commentsObjectList)))
    parsedComments = []
    listToParse = commentsObjectList[1] if specialCase else commentsObjectList
    for commentObj in listToParse:
        fechaObj = datetime.datetime.fromtimestamp(commentObj["tsMensaje"])
        fechaStr = fechaObj.strftime("%d/%m/%Y-%H:%M:%S")
        fechaArr = fechaStr.split("-")
        parsedComment = {
            "urlNoticia": urlNoticia,
            "fecha": fechaArr[0],
            "hora": fechaArr[1],
            "user": commentObj['usuarioOrigen'],
            "commentario": commentObj['contenido']
        }
        parsedComments.append(parsedComment)
    return parsedComments

In [56]:
parsedComments = extractComments(commentsResponse["mensajes"], url)

 	 -> parsing comments list with -121- elements:


In [57]:
parsedComments

[{'commentario': '<p>Tres millones de hectáreas de bosques arden en Rusia <a title="http://elpais.com/internacional/2019/08/01/actualidad/1564658691_230617.html" target="_blank" href="http://cort.as/-MEWg">http://cort.as/-MEWg</a></p>',
  'fecha': '01/08/2019',
  'hora': '15:08:56',
  'urlNoticia': 'https://elpais.com/internacional/2019/08/01/actualidad/1564658691_230617.html',
  'user': 'registrador'},
 {'commentario': '<p>Aún no toman en serio el calentamiento global. Las pérdidas económicas son mayores cada año. </p><p>Los gobiernos deben invertir para cambiar la matriz de energía, que aún sigue siendo petróleo, gas y carbón porque por más que crecen las energías verdes, cada año se necesita más y más energía: <a title="http://cort.as/9VGN" target="_blank" href="http://cort.as/9VGN">http://cort.as/9VGN</a> no hay que dejarlo al mercado, porque se está yendo más lento de lo que el planeta necesita. <a title="http://cort.as/-0X8m" target="_blank" href="http://cort.as/-0X8m">http://cor

In [58]:
len(parsedComments)

121